# Introduction
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

**What?** True multioutput regression

</font>
</div>

# Imports
<hr style = "border:2px solid black" ></hr>

In [3]:
from sklearn.datasets import make_regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

# Theoretical recalls
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>

- Many of the libraries offering tree-based regression model do have some sort multi-output capabilities.
- One of thing that is not clearly stated, in some cases, is that these API fit one model per output.
- Quoting from XGBoost: `Internally, XGBoost builds one model for each target similar to sklearn meta estimators, with the added benefit of reusing data and other integrated features like SHAP` 
- Quoting from ScikitLearn: `This strategy consists of fitting one regressor per target. This is a simple strategy for extending regressors that do not natively support multi-target regression.`
- **However**, to tell the whole story there is something called regressor chain which is able to explore correlations btw outputs. See the references for more.
- What if the we want to model a single model capable to fit more than one output? In this case we'll have to rely on NN which supports this natively.
- Multi-output regression can be supported directly by neural networks simply by specifying the number of target variables there are in the problem as the number of nodes in the output layer. 
- A task that has three output variables will require a neural network output layer with three nodes in the output layer, each with the linear (default) activation function.
    
</font>
</div>

# Create some synthetic data
<hr style = "border:2px solid black" ></hr>

In [2]:
# create dataset
X, y = make_regression(n_samples=1000, n_features=10,
                       n_informative=5, n_targets=3, random_state=2)
# summarize shape
print(X.shape, y.shape)

(1000, 10) (1000, 3)


In [4]:
def get_dataset():
    X, y = make_regression(n_samples=1000, n_features=10,
                           n_informative=5, n_targets=3, random_state=2)
    return X, y

# Create the model
<hr style = "border:2px solid black" ></hr>

In [8]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs,
              kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# CV
<hr style = "border:2px solid black" ></hr>

In [9]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    return results

In [10]:
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

2022-07-26 14:50:55.880139: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


>7.480
>7.674
>7.226
>7.231
>7.645
>7.755
>7.407
>7.382
>8.416
>7.793
>9.822
>8.077
>8.067
>8.873
>7.871
>8.134
>8.446
>7.451
>7.800
>9.576
>9.178
>8.274
>7.299
>6.895
>7.034
>7.789
>8.670
>8.341
>9.138
>6.856
MAE: 7.987 (0.757)


# Test the model
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>

- Once a model configuration is chosen, we can use it to fit a final model **on all available** data and make a prediction for new data.
- We can do this because we have already established the model is **not overfitting**.

</font>
</div>

In [12]:
# load dataset
X, y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
# make a prediction for new data
row = [-0.99859353, 2.19284309, -0.42632569, -0.21043258, -1.13655612, -
       0.55671602, -0.63169045, -0.87625098, -0.99445578, -0.3677487]
newX = np.asarray([row])
yhat = model.predict(newX)
print('Predicted: %s' % yhat[0])

Predicted: [-147.002    -73.88907  -94.83123]


# References
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

- [Deep Learning Models for Multi-Output Regression](https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/)
- [XGBoost and multioutput](https://xgboost.readthedocs.io/en/stable/tutorials/multioutput.html)
- [Scikit and multioutput](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html)
- [sklearn.multioutput.RegressorChain¶](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.RegressorChain.html)

</font>
</div>